In [1]:
import pandas as pd
import numpy as np

import os
import re
import csv
import glob
import os.path
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

from gensim import models
from gensim.models import CoherenceModel

In [2]:
stop_words = set(stopwords.words('english'))
stop_words.update(['http'] )

def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext


def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    cleaned = cleaned.replace("\s+"," ")

    return cleaned


def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def removeStopWords(sentence, stop_words):
    #global re_stop_words
    re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
    return re_stop_words.sub(" ", sentence)

### Import and text cleaning

In [4]:
# Import intermediate rditter dataframe
df_rd = pd.read_csv('df_rd_sentiment.csv')

# Clean text
stemmer = PorterStemmer()
df_rd['clean_text'] = df_rd.Title.str.lower()
df_rd['clean_text'] = df_rd['clean_text'].apply(cleanHtml)
df_rd['clean_text'] = df_rd['clean_text'].apply(cleanPunc)
df_rd['clean_text'] = df_rd['clean_text'].apply(keepAlpha)
df_rd['clean_text'] = df_rd['clean_text'].apply(lambda x: removeStopWords(x, stop_words))
df_rd['clean_text'] = df_rd['clean_text'].str.split().apply(lambda x: ' '.join(w for w in x if len(w)>2))
df_rd['clean_text'] = df_rd['clean_text'].str.split().apply(lambda x: ' '.join([stemmer.stem(w) for w in x]))

### Tokenisation and lemmatisation

In [5]:
from nltk.tokenize import sent_tokenize
df_rd['sentences'] = df_rd['clean_text'].apply(sent_tokenize)

from nltk.tokenize import word_tokenize
df_rd['tokens_sentences'] = df_rd['sentences'].apply(lambda sentences: [word_tokenize(sentence) for sentence in sentences])

from nltk import pos_tag
df_rd['POS_tokens'] = df_rd['tokens_sentences'].apply(lambda tokens_sentences: [pos_tag(tokens) for tokens in tokens_sentences])

# Inspired from https://stackoverflow.com/a/15590384
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatizing each word with its POS tag, in each sentence
df_rd['tokens_sentences_lemmatized'] = df_rd['POS_tokens'].apply(
    lambda list_tokens_POS: [
        [
            lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1])) 
            if get_wordnet_pos(el[1]) != '' else el[0] for el in tokens_POS
        ] 
        for tokens_POS in list_tokens_POS
    ]
)

from itertools import chain # to flatten list of sentences of tokens into list of tokens

df_rd['tokens'] = df_rd['tokens_sentences_lemmatized'].map(lambda sentences: chain.from_iterable(sentences))
df_rd['tokens'] = df_rd['tokens'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() 
                                                    and token.lower() not in stop_words and len(token)>1])

# Topic tagging

### Preparing a dictionary

In [6]:
# Preparing the corpus and the dictionary

from gensim import corpora
dictionary = corpora.Dictionary(df_rd.tokens)
dictionary.filter_extremes(no_below=20) #, no_above=0.15)
df_rd['corpus'] = [dictionary.doc2bow(tok) for tok in df_rd.tokens]
print(dictionary)

Dictionary(69 unique tokens: ['bitcoin', 'btc', 'everi', 'go', 'time']...)


### Preparing train and test datasets

In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_rd, random_state=20, test_size=0.20, shuffle=True)

print(train.shape)
print(test.shape)

(822, 23)
(206, 23)


### Training a model

In [9]:
# Define number of topics
num_topics = 3

passes = 10
lda_model = models.LdaModel(corpus=train['corpus'],\
                                  num_topics=num_topics, \
                                  id2word=dictionary, \
                                  random_state=100, \
                                  passes=passes, \
                                  #iterations=100,
                                  alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary.keys())
                                  #alpha='auto', \
                                  #eta='auto'
                                  #per_word_topics=True
                                 )

### Evaluate performance

In [10]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=test['tokens'], dictionary=dictionary, coherence='c_v')

print('Coherence per topic:')
print(coherence_model_lda.get_coherence_per_topic())
print()
print('==========================================')
print('Topics as word lists:')
print(coherence_model_lda.top_topics_as_word_lists(lda_model, dictionary, topn=20))
print()
print('==========================================')
print('Topics:')
print(coherence_model_lda.topics)
print()
print('==========================================')
print('Performance on the test set:')
# Compute Perplexity
#print('Perplexity: ', lda_model.log_perplexity(test['corpus']))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=test['tokens'], dictionary=dictionary, coherence='c_v')
coherence_list = coherence_model_lda.get_coherence_per_topic()
print('Mean coherence of the model:      {}'.format(coherence_model_lda.get_coherence()))
print('Median coherence of the model:    {}'.format(np.median(coherence_list)))
print('Maximal coherence of the model:   {}'.format(max(coherence_list)))
print('Minimal coherence of the model:   {}'.format(min(coherence_list)))

Coherence per topic:
[0.5797160822696608, 0.5588739986067579, 0.4173680889069179]

Topics as word lists:
[['bitcoin', 'bch', 'get', 'buy', 'btc', 'year', 'sell', 'like', 'today', 'time', 'accept', 'first', 'guy', 'tri', 'go', 'thank', 'block', 'stop', 'post', 'think'], ['btc', 'use', 'crypto', 'like', 'bitcoin', 'peopl', 'pay', 'see', 'coin', 'need', 'support', 'fee', 'one', 'time', 'user', 'give', 'abc', 'want', 'high', 'dont'], ['bitcoin', 'cash', 'bch', 'bank', 'make', 'new', 'day', 'peopl', 'fee', 'com', 'use', 'money', 'btc', 'transact', 'one', 'best', 'currenc', 'world', 'feel', 'work']]

Topics:
[array([ 0,  5, 40,  9,  1, 43,  8, 15, 27,  4, 37,  7, 13, 28,  3, 57, 50,
       14, 60, 20]), array([ 1, 49, 35, 15,  0, 33, 44, 56, 25, 59, 51, 18, 22,  4, 42, 66, 68,
       21, 26, 58]), array([ 0, 34,  5, 10, 53, 38, 54, 33, 18,  6, 49, 32,  1, 36, 22, 29, 16,
       45, 31, 17])]

Performance on the test set:
Mean coherence of the model:      0.5186527232611122
Median coherence o

### Visualisation

In [12]:
import pyLDAvis
import pyLDAvis.gensim
data_ = pyLDAvis.gensim.prepare(lda_model, df_rd['corpus'], dictionary)
pyLDAvis.display(data_)

/Users/snezana/testenv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Applying topic tagging

In [13]:
from operator import itemgetter

def get_dominant_topic(l):
    return max(l,key=itemgetter(1))[0]

df_rd['categorisation_full'] = df_rd['corpus'].apply(lambda x: lda_model.get_document_topics(x, minimum_probability=0))
df_rd['rd_topic'] = df_rd['categorisation_full'].apply(lambda x: get_dominant_topic(x))

df_rd.drop(columns=['rd_sentiments', 'clean_text', 'tokens_sentences', 'tokens', 'POS_tokens', 'sentences',
       'tokens_sentences_lemmatized', 'corpus', 'categorisation_full'], inplace=True)

/Users/snezana/testenv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Save the df to csv

In [15]:
df_rd.to_csv('df_rd_topic.csv', index=False)

/Users/snezana/testenv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
